In [ ]:
!pip install pytorch_fid 

In [ ]:
# FID
import os
from pytorch_fid import fid_score

# Đường dẫn đến các thư mục chứa ảnh
real_images_path = '/kaggle/input/image22/Image2'  # Thư mục chứa ảnh thật
generated_images_path= '/kaggle/input/200-epochs-poisson-output'  # Thư mục chứa ảnh sinh ra


# Tính FID
fid_value = fid_score.calculate_fid_given_paths([real_images_path, generated_images_path], batch_size=50, device='cuda', dims=2048)
print(f"FID score : {fid_value}")

In [ ]:
import os
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchmetrics.image import StructuralSimilarityIndexMeasure

# Khởi tạo metric SSIM
ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0)

def calculate_ssim(imageA, imageB):
    # Chuyển đổi ảnh sang tensor và chuẩn hóa về khoảng [0, 1]
    transform = transforms.ToTensor()
    imageA = transform(imageA).unsqueeze(0)
    imageB = transform(imageB).unsqueeze(0)
    
    # Tính SSIM
    ssim_value = ssim_metric(imageA, imageB)
    return ssim_value.item()

def calculate_average_ssim(original_folder, generated_folder):
    total_ssim = 0
    count = 0
    
    for filename in os.listdir(original_folder):
        # Tạo tên tệp tương ứng trong thư mục generated
        base_name, ext = os.path.splitext(filename)
        original_path = os.path.join(original_folder, filename)
        generated_path = os.path.join(generated_folder, f"{base_name}.png")  # Giả sử ảnh generated là .png
        
        if os.path.isfile(original_path) and os.path.isfile(generated_path):
            # Đọc ảnh
            imageA = Image.open(original_path).convert('RGB')
            imageB = Image.open(generated_path).convert('RGB')
            
            # Thay đổi kích thước ảnh trong generated_folder cho khớp với ảnh trong original_folder
            if imageA.size != imageB.size:
                imageB = imageB.resize(imageA.size)
            
            # Tính SSIM và thêm vào tổng SSIM
            ssim_score = calculate_ssim(imageA, imageB)
            total_ssim += ssim_score
            count += 1
        else:
            print(f"Ảnh {filename} không tồn tại trong một trong hai thư mục.")
    
    # Tính SSIM trung bình
    average_ssim = total_ssim / count if count > 0 else 0
    return average_ssim

# Đường dẫn đến thư mục ảnh gốc và thư mục ảnh mô hình tạo ra
original_folder = '/kaggle/input/image22/Image2'
generated_folder = '/kaggle/input/200-epochs-poisson-output'

# Tính SSIM trung bình
average_ssim = calculate_average_ssim(original_folder, generated_folder)
print("SSIM trung bình:", average_ssim)

In [ ]:
pip install torchmetrics[image]

In [ ]:
pip install torch torchmetrics pillow

In [ ]:
import torch
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
from PIL import Image
import os
import torchvision.transforms as transforms

# Khởi tạo LPIPS metric với mạng nền tảng 'alex'
lpips = LearnedPerceptualImagePatchSimilarity(net_type='alex')

# Đường dẫn đến thư mục ảnh gốc (jpg) và ảnh đã tạo (png)
original_folder = '/kaggle/input/image22/Image2'
generated_folder = '/kaggle/input/200-epochs-poisson-output'

# Transform để chuyển ảnh sang tensor và chuẩn hóa trong khoảng [-1, 1]
transform = transforms.Compose([
    transforms.Resize((100, 100)),  # Điều chỉnh kích thước nếu cần
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Chuẩn hóa để ảnh nằm trong khoảng [-1, 1]
])

# Duyệt qua từng ảnh trong thư mục
scores = []
for filename in os.listdir(original_folder):
    if filename.endswith('.jpg'):
        original_path = os.path.join(original_folder, filename)
        generated_path = os.path.join(generated_folder, filename.replace('.jpg', '.png'))

        # Đảm bảo ảnh generated tương ứng tồn tại
        if os.path.exists(generated_path):
            # Đọc và tiền xử lý ảnh
            original_image = transform(Image.open(original_path).convert('RGB'))
            generated_image = transform(Image.open(generated_path).convert('RGB'))

            # Thêm batch dimension
            original_image = original_image.unsqueeze(0)
            generated_image = generated_image.unsqueeze(0)

            # Tính LPIPS cho cặp ảnh
            score = lpips(original_image, generated_image)
            scores.append(score.item())

            print(f"LPIPS score for {filename}: {score.item()}")

# Tính LPIPS trung bình
if scores:
    avg_score = sum(scores) / len(scores)
    print(f"Average LPIPS score: {avg_score}")
else:
    print("No valid image pairs found.")

In [ ]:
pip install tensorflow numpy scikit-learn


In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from sklearn.metrics.pairwise import polynomial_kernel

# Hàm đọc ảnh từ folder
def load_images_from_folder(folder, image_size=(384, 512)):
    images = []
    for filename in os.listdir(folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path).convert('RGB')  # Đảm bảo ảnh là RGB
            img = img.resize(image_size)  # Resize nếu cần (384x512 là kích thước gốc)
            images.append(np.array(img))
    return np.array(images)

# Hàm trích xuất đặc trưng từ Inception v3
def extract_features(images, model):
    images = preprocess_input(images)  # Tiền xử lý ảnh
    features = model.predict(images, batch_size=32, verbose=0)  # Dự đoán đặc trưng
    return features

# Hàm tính toán Kernel Inception Distance (KID)
def calculate_kid(real_features, generated_features):
    # Kernel giữa các đặc trưng
    real_kernel = polynomial_kernel(real_features)
    gen_kernel = polynomial_kernel(generated_features)
    mix_kernel = polynomial_kernel(real_features, generated_features)
    
    # Tính toán KID
    kid = np.mean(real_kernel) + np.mean(gen_kernel) - 2 * np.mean(mix_kernel)
    return kid

# Đường dẫn tới folder ảnh
real_images_folder = "/kaggle/input/image22/Image2"
gen_images_folder = "/kaggle/input/200-epochs-poisson-output"

# Tải dữ liệu ảnh từ folder
real_images = load_images_from_folder(real_images_folder)
gen_images = load_images_from_folder(gen_images_folder)

# Đảm bảo rằng ảnh có kích thước (299, 299, 3) để phù hợp với InceptionV3
real_images_resized = np.array([np.array(Image.fromarray(img).resize((299, 299))) for img in real_images])
gen_images_resized = np.array([np.array(Image.fromarray(img).resize((299, 299))) for img in gen_images])

# Tải mô hình Inception v3
inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

# Trích xuất đặc trưng
real_features = extract_features(real_images_resized, inception_model)
gen_features = extract_features(gen_images_resized, inception_model)

# Tính chỉ số KID
kid_score = calculate_kid(real_features, gen_features)
print(f"Chỉ số KID: {kid_score}")
